Outline


- How to define and create types (`struct`/`mutable struct`)
- How to create parametric types.
- How to define restrictions on types during construction.
- How to define types with default values.
- How to make custom prints


## Defining your own type (struct/mutable struct)


Types can be thought as "boxes" containing data. Each "box" is called  **``field``** and can contain any  **``Type``** inside, each field is identified by a name filed. In order to see the different fields of a type you can use the function **``fieldnames``**.

- Each field can have a type, the symbol **`::`** is used to assign a type to a fieldname.
    - For example **`x::Float64`** states that `x` has type ``Float64``.
- Each field can have a different type

In [60]:
using Revise

┌ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1189


In [131]:
mutable struct Position
    x::Float64
    y::Float64
    z::Float64
end
p = Position(10,10,2)

Position(10.0, 10.0, 2.0)

In [11]:
p.x = 23

23

In [12]:
p

Position(23.0, 10.0, 2.0)

## Inmutable Types

As the name suggests, you cannot mutate the fieldnames of **``immutable``** types.


**An immutable object might contain mutable objects**, such as arrays, as fields. Those contained objects will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

In [133]:
struct Point
    x::Float64
    y::Float64
    z::Float64
end

p = Point(10,10,2)

ErrorException: invalid redefinition of constant Point

In [20]:
p.x = 23

ErrorException: type Point is immutable

In [24]:
struct VectorPoint
    x::Array{Float64}
    y::Array{Float64}
    z::Array{Float64}
end
p = VectorPoint([10,2],[10,1],[10,10])

VectorPoint([10.0, 2.0], [10.0, 1.0], [10.0, 10.0])

In [29]:
p.x = [3.,1.]

ErrorException: type VectorPoint is immutable

In [31]:
p.x .= [3.,1.]

2-element Array{Float64,1}:
 3.0
 1.0

In [32]:
p

VectorPoint([3.0, 1.0], [10.0, 1.0], [10.0, 10.0])

### Immutable types "can change"

If a field of an immutable type can change (for example, an array can grow) then the instanciated immutable type can also change.

Let us see it in a simple case, we have a `Family` type with field `members` which is an `Array`. An instanciated `Family` can grow in any point even though the type `Family` has beeen defined as `immutable`.

In [35]:

struct Family
    members::Array{String}
    name::String
end

In [36]:
simpsons = Family(["Lisa","Hommer","Bart","Marge"],"Simpson")

Family(["Lisa", "Hommer", "Bart", "Marge"], "Simpson")

In [37]:
push!(simpsons.members, "maggie")

5-element Array{String,1}:
 "Lisa"  
 "Hommer"
 "Bart"  
 "Marge" 
 "maggie"

In [38]:
simpsons

Family(["Lisa", "Hommer", "Bart", "Marge", "maggie"], "Simpson")

What we can't do is rewrite `simpsons.members`  field with another instance of `Array`.

In [39]:
# this is not allowed
simpsons.members =  ["Mariano", "Melindro"]

ErrorException: type Family is immutable

In [40]:
# This is allowed because we are changing the object inplace
simpsons.members .=  ["L" , "H",  "B" ,  "M", "L"]

5-element Array{String,1}:
 "L"
 "H"
 "B"
 "M"
 "L"

#### Another example of immutable type

- We cannot do  `model.Θ = model.Θ + 1` since it would change the array.


- We can simply do `model.Θ .= model.Θ + 1`


In [45]:
struct Model
    name::String
    Θ::Array
end

In [52]:
model = Model("Percepton", zeros(10))

Model("Percepton", [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [53]:
model.Θ[1] = 12

12

In [54]:
model.Θ = model.Θ .+ 1

ErrorException: type Model is immutable

In [55]:
model.Θ .= model.Θ .+ 1

10-element Array{Float64,1}:
 13.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0

## Parametric types

To make the code more general we can use ** parametric types** which define a family of types by a parameter. 

- Parametric types can depend on multiple parameters `Point{T1,T2}`.


In [61]:
struct PositionTyped{T<:Number}
    x::T
    y::T
    z::T
end

In [62]:
p = PositionTyped(10,10,2)

PositionTyped{Int64}(10, 10, 2)

In [64]:
p = PositionTyped(10.,10.,2.)

PositionTyped{Float64}(10.0, 10.0, 2.0)

In [82]:
p = PositionTyped{Float32}(10,10,2)

PositionTyped{Float32}(10.0f0, 10.0f0, 2.0f0)

In [87]:
p = PositionTyped{Float32}()

MethodError: MethodError: no method matching PositionTyped{Float32}()
Closest candidates are:
  PositionTyped{Float32}(!Matched::Any, !Matched::Any, !Matched::Any) where T<:Number at In[61]:3

#### This code
``` Julia
struct PositionTyped{T}
    x::T
    y::T
    z::T
end
```

#### Already defines all this


``` Julia
struct PositionTyped{T<:Any}
  x::T
  y::T
  z::T
end

PositionTyped{T<:Any}(x::T, y::T, z::T) = Position{T}(x,y,z)
```

### Custom constructor

In [106]:
struct PositionTyped2{T}
  x::T
  y::T
  z::T
end

# You do not need to write this, this is done automatically
PositionTyped2{T}() where {T<:Number} = PositionTyped2{T}(0,0,0)

In [110]:
p = PositionTyped2{Float32}()

PositionTyped2{Float32}(0.0f0, 0.0f0, 0.0f0)

In [111]:
p = PositionTyped2{Float64}()

PositionTyped2{Float64}(0.0, 0.0, 0.0)

In [112]:
p = PositionTyped2{}()

UndefVarError: UndefVarError: T not defined

If we want to be able to instanciate a `PositionTyped2` without specifying the type
we need to create a constructor

In [122]:
PositionTyped2() = PositionTyped2(0,0,0)

PositionTyped2

In [123]:
PositionTyped2{Int64}()

PositionTyped2{Int64}(0, 0, 0)

In [121]:
PositionTyped2{Int32}()

PositionTyped2{Int32}(0, 0, 0)

#### Parametric types can depend on more than one type

In [89]:
struct Position{T1,T2}
    x::T1
    y::T1
    z::T2
end

In [90]:
Position(2,3,123.)

Position{Int64,Float64}(2, 3, 123.0)

# Set restrictions on custom types


Sometimes we don't want to have types that are too general and can be missused by the end user in unnexpected ways. In order to avoid this we can put restrictions.

#### Position example: Coordinates should be Numbers

For example, our `Position` type can be created to be used only with numbers inside its fieldnames, nevertheless, our previous definition

``` Julia
type Position{T}
    x::T
    y::T
    z::T
end
```

can accept any `Type` inside the fields `x`, `y` and `z` as long as it's the same type for all fieldnames. 

``` Julia
Point("This", "is", "unnexpected")
```
```
Point{String}("This","is","unnexpected")
```

#### Using the subtype operator <:

The operator **`<:`** allows us to verify if a `Type` is subtype of an `AbstractType`.


In [91]:
Int64 <: Real

true

In [92]:
mutable struct PositionUnrestricted
    x
    y
    z
end

In [93]:
PositionUnrestricted("This", "is", "unnexpected")

PositionUnrestricted("This", "is", "unnexpected")

In [94]:
mutable struct PositionRestricted{T<:Real}
    x::T
    y::T
    z::T
end

In [96]:
# Now the following Position with fields taking String values is not accepted
PositionRestricted("This", "is", "unnexpected")

MethodError: MethodError: no method matching PositionRestricted(::String, ::String, ::String)

## Set default values for a custom type

Constructors can be used to generate ways to construct custom types. This could be usefull, for example, in order to set default values to your custom types.

In [97]:
mutable struct PositionWithDefault{T}
    x::T
    y::T
    z::T
end

# Make a constructor that sets some default values 
PositionWithDefault(x,y) = Position(x,y,0)

# Make a constructor that sets all values as default
PositionWithDefault() = Position(0,0,0)

PositionWithDefault

In [100]:
PositionWithDefault(10, 10)

Position{Int64,Int64}(10, 10, 0)

In [105]:
PositionWithDefault()

Position{Int64,Int64}(0, 0, 0)

Obviously we can add `Position(x,y) = Position(x,y,0)` as a new constructor without actually creating a new struct `PositionWithDefault`.

In [102]:
Position(10, 10)

MethodError: MethodError: no method matching Position(::Int64, ::Int64)
Closest candidates are:
  Position(::T1, ::T1, !Matched::T2) where {T1, T2} at In[89]:2

In [103]:
Position(x,y) = Position(x,y,0)

Position

In [104]:
Position(10, 10)

Position{Int64,Int64}(10, 10, 0)

# Set restrictions for custom types, inner constructors

**We can enforce properties for fields of a custom type in its construction** using inner constructors with restrictions.

We can add a function inside a type used to instanciate the type.

This function can be used for example in order to..

- Put restrictions on the type, such as..
     - not allow negative values 
     - Not allow 'rare' characters

In [156]:
mutable struct OrderedPair
  x::Real
  y::Real
  OrderedPair(x,y) = x > y ? error("out of order, x=$x should be bigger than y=$y") : new(x,y)
end

In [157]:
OrderedPair(1,10)

OrderedPair(1, 10)

In [159]:
# This is not a valid ordered pair
OrderedPair(10,1)

ErrorException: out of order, x=10 should be bigger than y=1

## Parametric types with restrictions
#### inner constructor for parametric types => outer constructor needed

In [2]:
struct OrderedPairParametric{T}
  x::T
  y::T 
  OrderedPairParametric(x::T,y::T) where {T<:Any} = x > y ? error("x=$x should be bigger than y=$y") : new{T}(x,y)
end

# THIS WILL NOT WORK we need an outer constructor if the type contains functions
# Notice that 
#     OrderedPair(x,y) = x > y ? error("out of order") : new(x,y) 
# is a function
OrderedPairParametric(1,10)

OrderedPairParametric{Int64}(1, 10)

In [8]:
x = OrderedPairParametric(1.,10.)

OrderedPairParametric{Float64}(1.0, 10.0)

In [4]:
OrderedPairParametric(13.,10.)

ErrorException: x=13.0 should be bigger than y=10.0

In [183]:
OrderedPairParametric()

MethodError: MethodError: no method matching OrderedPairParametric()
Closest candidates are:
  OrderedPairParametric(!Matched::T, !Matched::T) where T at In[180]:4
  OrderedPairParametric(!Matched::Any, !Matched::Any) at In[170]:4

In [186]:
OrderedPairParametric() = OrderedPairParametric(0,1)
OrderedPairParametric()

OrderedPairParametric{Int64}(0, 1)

## Make custom print for a custom type

Sometimes a `struct might contain too much information`. We can make a custom `show` method for a struct that allows us to define what will be shown in the REPL when we `print(x)` for `x::T`.


In [14]:
import Base.show

In [15]:
x

OrderedPairParametric: x=1.0, y=10.0

In [16]:
Base.show(io::IO, p::OrderedPairParametric) = print(io, "OrderedPairParametric: x=$(p.x), y=$(p.y)")

In [17]:
x

OrderedPairParametric: x=1.0, y=10.0

In [18]:
print(x)

OrderedPairParametric: x=1.0, y=10.0